<a href="https://colab.research.google.com/github/raffieeey/MasterResearchAutoML/blob/master/h2oautoml/02A_H2OAutoml_Regres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing dependecies for H2O automl

In [9]:
!pip install pmlb
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install pytictoc

In [0]:
#!pip uninstall h2o -y

## Installing H2O

In [11]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


## Getting the number of Cores

In [12]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa itlb_multihit
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits 

##Starting H2O

In [0]:
import h2o

In [14]:
h2o.init(nthreads=2,
                 min_mem_size="2048M",
                 max_mem_size="2048M")

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 min 07 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_qy40d6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.000 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


## Creating all the metrics for Regression algorithm

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
import math
import numpy as np

class MetricsCalc:

    def __init__(self, truth, predictions):
        self.truth = truth.as_data_frame()
        self.predictions = predictions
        self._type = 'regression'

    def mae(self):
        return float(mean_absolute_error(self.truth, self.predictions))

    def mse(self):
        return float(mean_squared_error(self.truth, self.predictions))

    def msle(self):
      try:
        return float(mean_squared_log_error(self.truth, self.predictions))
      except:
        return np.nan
    
    def rmse(self):
        return math.sqrt(self.mse())

    def rmsle(self):
        return math.sqrt(self.msle())

    def r2(self):
        return float(r2_score(self.truth, self.predictions))

## Running H2O autoML with 
- 120 seconds
- Seed 69
- 2 Cores
- 2GB memory

- Tested for metrics: 
> - Deviance
> - Mean Squared error
> - Root Mean Squared error
> - Mean Absolute error
> - Root Mean Squared Logarithmic Error

- Recorded for metrics: 
> - Mean Absolute error
> - Mean Squared error
> - Mean Squared Logarithmic Error
> - Root Mean Squared error
> - Root Mean Squared Logarithmic Error
> - r2

In [16]:
from pmlb import classification_dataset_names, regression_dataset_names,fetch_data
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from pytictoc import TicToc
import pandas as pd
from pprint import pprint

df_result = pd.DataFrame(columns=["dataset_name", "metrics_recorded", "MAE", "MSE","MSLE", "RMSE", "RMSLE", "r2"])
preds = {}
top_algo = {}
t = TicToc()


sel_clss_dtst = ['adult','agaricus-lepiota', 'churn', 'nursery', 'satimage','texture']
sel_rgrs_dtst = ['294_satellite_image','218_house_8L', '227_cpu_small', '503_wind', '344_mv','215_2dplanes']

ix = 0

for rgrs_dtst in sel_rgrs_dtst:
  for metrics_recorded in ['RMSLE', 'MAE', 'RMSE', 'MSE', 'deviance']:
        df_result.loc[ix,'dataset_name'] = rgrs_dtst
        df_result.loc[ix,'metrics_recorded'] = metrics_recorded
        print("_"*80)
        print(f"*** Dataset Name: {rgrs_dtst} ***")
        X, y = fetch_data(rgrs_dtst, return_X_y=True)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=69)
        X_train = h2o.H2OFrame(X_train)
        X_test = h2o.H2OFrame(X_test)
        y_train = h2o.H2OFrame(y_train)
        y_test = h2o.H2OFrame(y_test)
        aml = H2OAutoML(seed=69, max_runtime_secs=120,sort_metric=metrics_recorded)
        t.tic()
        aml.train(x=X_train.columns, y=X_train.cbind(y_train)[:,-1].columns[0], training_frame=X_train.cbind(y_train))
        t.toc()    
        lb = aml.leaderboard
        top_algo[rgrs_dtst] = aml.leader.actual_params
        print("*** Top algorithm: ***", lb.head(5))
        pprint(aml.leader.actual_params)
        preds[rgrs_dtst] = aml.leader.predict(X_test).as_data_frame()
        p = MetricsCalc(y_test,preds[rgrs_dtst])
        df_result.loc[ix,'MAE'] = p.mae()
        df_result.loc[ix,'MSE'] = p.mse()
        df_result.loc[ix,'MSLE'] = p.msle()
        df_result.loc[ix,'RMSE'] = p.rmse()
        df_result.loc[ix,'RMSLE'] = p.rmsle()
        df_result.loc[ix,'r2'] = p.r2()
        ix+=1

________________________________________________________________________________
*** Dataset Name: 294_satellite_image ***
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
Elapsed time is 123.759938 seconds.
*** Top algorithm: *** 

model_id,rmsle,mean_residual_deviance,rmse,mse,mae
StackedEnsemble_AllModels_AutoML_20200525_103431,0.141452,0.455376,0.674815,0.455376,0.367744
StackedEnsemble_BestOfFamily_AutoML_20200525_103431,0.144813,0.471991,0.687016,0.471991,0.378717
GBM_grid__1_AutoML_20200525_103431_model_1,0.154935,0.51656,0.718721,0.51656,0.411644
XGBoost_grid__1_AutoML_20200525_103431_model_1,0.155175,0.533355,0.730311,0.533355,0.397317
XGBoost_2_AutoML_20200525_103431,0.162738,0.574378,0.757877,0.574378,0.426047



{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'GBM_grid__1_AutoML_20200525_103431_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_grid__1_AutoML_20200525_103431_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_2_AutoML_20200525_103431',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
   

model_id,mae,mean_residual_deviance,rmse,mse,rmsle
StackedEnsemble_AllModels_AutoML_20200525_103638,0.370162,0.456296,0.675497,0.456296,0.142107
StackedEnsemble_BestOfFamily_AutoML_20200525_103638,0.374735,0.467617,0.683825,0.467617,0.14399
DRF_1_AutoML_20200525_103638,0.387627,0.60322,0.776672,0.60322,0.166207
XRT_1_AutoML_20200525_103638,0.38846,0.606091,0.778518,0.606091,0.16763
XGBoost_grid__1_AutoML_20200525_103638_model_1,0.397194,0.533058,0.730108,0.533058,0.155151



{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'DRF_1_AutoML_20200525_103638',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XRT_1_AutoML_20200525_103638',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_grid__1_AutoML_20200525_103638_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                     

...


{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_3_AutoML_20200525_113132',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'DeepLearning_grid__1_AutoML_20200525_113132_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_grid__1_AutoML_20200525_113132_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'Ke

model_id,mse,mean_residual_deviance,rmse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200525_113338,1.02297,1.02297,1.01142,0.808276,nan
StackedEnsemble_BestOfFamily_AutoML_20200525_113338,1.02924,1.02924,1.01451,0.810574,nan
XGBoost_3_AutoML_20200525_113338,1.06728,1.06728,1.03309,0.825466,nan
DeepLearning_grid__1_AutoML_20200525_113338_model_1,1.08517,1.08517,1.04171,0.830976,nan
XGBoost_grid__1_AutoML_20200525_113338_model_1,1.09844,1.09844,1.04807,0.836679,nan



{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_3_AutoML_20200525_113338',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'DeepLearning_grid__1_AutoML_20200525_113338_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_grid__1_AutoML_20200525_113338_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'Ke

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200525_113544,1.02284,1.01135,1.02284,0.807717,nan
StackedEnsemble_BestOfFamily_AutoML_20200525_113544,1.0275,1.01366,1.0275,0.809462,nan
XGBoost_3_AutoML_20200525_113544,1.06341,1.03122,1.06341,0.823911,nan
XGBoost_grid__1_AutoML_20200525_113544_model_1,1.09718,1.04746,1.09718,0.836349,nan
DeepLearning_1_AutoML_20200525_113544,1.12554,1.06092,1.12554,0.846209,nan



{'base_models': [{'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_3_AutoML_20200525_113544',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'XGBoost_grid__1_AutoML_20200525_113544_model_1',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
                             'schema_type': 'Key<Keyed>',
                             'schema_version': 3},
                  'name': 'DeepLearning_1_AutoML_20200525_113544',
                  'type': 'Key<Keyed>'},
                 {'URL': None,
                  '__meta': {'schema_name': 'KeyV3',
        

## Looking at the predictions

In [17]:
preds

{'215_2dplanes':        predict
 0    -1.287392
 1    -0.158216
 2    -7.255321
 3    -3.209462
 4    -5.210870
 ...        ...
 4072  4.119123
 4073  0.879709
 4074  0.916403
 4075  9.042642
 4076  0.989254
 
 [4077 rows x 1 columns], '218_house_8L':            predict
 0     60408.557920
 1     27888.735944
 2     18627.754034
 3     33462.166310
 4     21645.831742
 ...            ...
 2274  39089.203845
 2275  57368.499964
 2276  19410.952009
 2277  34211.909716
 2278  34094.494252
 
 [2279 rows x 1 columns], '227_cpu_small':        predict
 0    94.780567
 1    90.028973
 2    84.798111
 3    94.564485
 4    91.729682
 ..         ...
 815  90.610706
 816  94.010542
 817  88.696018
 818  94.524263
 819  87.328982
 
 [820 rows x 1 columns], '294_satellite_image':       predict
 0    7.109498
 1    0.963397
 2    1.016207
 3    6.264841
 4    5.052154
 ..        ...
 639  0.909742
 640  0.854121
 641  2.241626
 642  6.929826
 643  5.993401
 
 [644 rows x 1 columns], '344_mv':        

## Saving the predictions

In [0]:
for i, val in enumerate(preds):
  preds[val].to_csv(f"{val}.csv")

## Looking at the AUC

In [19]:
df_result

,dataset_name,metrics_recorded,MAE,MSE,MSLE,RMSE,RMSLE,r2
0,294_satellite_image,RMSLE,0.359649,0.470867,0.0194859,0.686198,0.139592,0.902675
1,294_satellite_image,MAE,0.364449,0.478975,0.019771,0.69208,0.140609,0.901
2,294_satellite_image,RMSE,0.361081,0.471356,0.0195324,0.686554,0.139758,0.902574
3,294_satellite_image,MSE,0.364208,0.478757,0.0194983,0.691923,0.139636,0.901045
4,294_satellite_image,deviance,0.359877,0.470677,0.0195597,0.686059,0.139856,0.902715
5,218_house_8L,RMSLE,15664.1,8.37286e+08,0.375138,28935.9,0.612485,0.676499
6,218_house_8L,MAE,15621.5,8.37814e+08,0.376622,28945,0.613695,0.676295
7,218_house_8L,RMSE,15637.9,8.35226e+08,0.380183,28900.3,0.61659,0.677295
8,218_house_8L,MSE,15599.5,8.34674e+08,0.376506,28890.7,0.613601,0.677508
9,218_house_8L,deviance,15608.4,8.34324e+08,0.377316,28884.7,0.614261,0.677643


## Looking at the mean per class error

## Looking at the top algorithm used by the AutoML

In [20]:
top_algo

{'215_2dplanes': {'base_models': [{'URL': None,
    '__meta': {'schema_name': 'KeyV3',
     'schema_type': 'Key<Keyed>',
     'schema_version': 3},
    'name': 'XGBoost_3_AutoML_20200525_113544',
    'type': 'Key<Keyed>'},
   {'URL': None,
    '__meta': {'schema_name': 'KeyV3',
     'schema_type': 'Key<Keyed>',
     'schema_version': 3},
    'name': 'XGBoost_grid__1_AutoML_20200525_113544_model_1',
    'type': 'Key<Keyed>'},
   {'URL': None,
    '__meta': {'schema_name': 'KeyV3',
     'schema_type': 'Key<Keyed>',
     'schema_version': 3},
    'name': 'DeepLearning_1_AutoML_20200525_113544',
    'type': 'Key<Keyed>'},
   {'URL': None,
    '__meta': {'schema_name': 'KeyV3',
     'schema_type': 'Key<Keyed>',
     'schema_version': 3},
    'name': 'DRF_1_AutoML_20200525_113544',
    'type': 'Key<Keyed>'},
   {'URL': None,
    '__meta': {'schema_name': 'KeyV3',
     'schema_type': 'Key<Keyed>',
     'schema_version': 3},
    'name': 'XGBoost_1_AutoML_20200525_113544',
    'type': 'Key<Keye